# MDP

Когда Инокетений Вальдемарович Носочков приходит на работу, то с вероятностью 50% он хочет пойти в бар, с 10% – покушац, с 20% – спать. Если он ничего не хочет, то он продолжает работать. Когда Кеша пьет в Баре, то в 10% посещений он возвращается на работу и с вероятностью 30% идет спать, но в остальное время продолжает пить. Когда он просыпается, то с вероятностью 40% идет покушац и с вероятностью 60% идет в бар пить дальше. Если вдруг г-н Носочков поел, то с вероятностью 80% он начинает работать, а если не срослось с работой, то он начинает хотеть спать.

Определите вероятности, что наш герой прямо сейчас работает, пьет в баре, спит или ест, при условии, что если Инокетений чего-то хочет, то делает это.

# Practice: gym interface and crossentropy method

_Reference:_ This notebook is based on Practical RL [week01](https://github.com/yandexdataschool/Practical_RL/tree/master/week01_intro)

In [ ]:
# !pip install requirements.txt

## OpenAI Gym

Следующие несколько недель мы потратим на изучение алгоритмов, которые помогают в процессе принятия решений. Затем нам понадобится несколько интересных задач для тестирования наших алгоритмов.

Вот тут-то и вступает в игру OpenAI Gym. Это библиотека Python, которая охватывает множество классических задач, связанных с принятием решений, включая управление роботами, видеоигры и настольные игры.

Итак, вот как это работает:

In [ ]:
import gymnasium as gym
gym.__version__

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
env = gym.make("MountainCar-v0", render_mode='rgb_array')
env.reset()

env_screen = env.render()
plt.imshow(env_screen)
plt.show()

print("Observation space:", env.observation_space)
print("Action space:", env.action_space)



### Gym interface

Существуют три основных метода настройки среды:
* `reset()`: возвращает среду в исходное состояние и _return (возвращает ее обратно)_
* `render()`: показывает текущее состояние среды (более красочная версия).
* `step(a)`: совершаем действие "a" и возвращаем "(new_state, вознаграждение, is_done, информация)"
* `new_state`: новое состояние сразу после совершения действия "a".
* `reward`: число, представляющее вашу награду за совершение действия "a"
* `is_done`: значение True, если MDP только что завершился, и значение False, если все еще продолжается
* `info`: некоторые дополнительные сведения о том, что только что произошло. Пока не обращайте на это внимания.

In [ ]:
state = env.reset()
print("initial state:", state)

В горной машине наблюдение - это всего лишь две цифры: положение автомобиля и скорость.

Давайте выполним действие 2, которое расшифровывается как "поверни направо".

In [ ]:
print("taking action 2 (right)")
new_state, reward, is_done, truncated, _ = env.step(2)

print("new state:", new_state)
print("reward:", reward)
print("is game over?:", is_done)
print("is game truncated due to time limit?:", is_done)

### Play with it

Ниже приведен код, который поворачивает автомобиль вправо. Однако, если вы просто используете политику по умолчанию, автомобиль не достигнет крайнего правого флажка из-за силы тяжести.

__Ваша задача__ - исправить это. Найдите стратегию, которая позволит добраться до флага. 

На данный момент от вас не требуется создавать какие-либо сложные алгоритмы, и вам определенно не нужно знать какие-либо методы обучения с подкреплением для этого. Не стесняйтесь использовать жесткий код :)

In [ ]:
actions = {"left": 0, "stop": 1, "right": 2}

def policy(state, time_step):
    # Write the code for your policy here. You can use the current state
    # (a tuple of position and velocity), the current time step, or both,
    # if you want.
    position, velocity = state
    
    # Ваш код здесь

    return actions["right"]

In [ ]:
from IPython.display import clear_output, display
time_limit = 250
is_done = 0

state, _ = env.reset()
for time_step in range(time_limit):
    plt.gca().clear()
    # Choose action based on your policy.

    # Pass the action to the environment.
    
    # We don't do anything with reward here because MountainCar is a very
    # simple environment, and reward is a constant -1 (meaning that your
    # goal is to end the episode as quickly as possible).
    action = policy(state, time_step)
    state, reward, is_done, truncated, _ = env.step(action)

    # Draw game image on display.
    
    plt.imshow(env.render())

    display(plt.gcf())
    clear_output(wait=True)

    if is_done:
          print("Well done!")
          break
if not is_done:
    print("Time limit exceeded. Try again.")

## Crossentropy method

Теперь, когда мы знаем, как работает `gym`, давайте попробуем решить более сложную задачу, используя метод перекрестной энтропии.

In [ ]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
env.reset()
plt.imshow(env.render())


Поскольку `Taxi-v3` представляет собой гораздо более сложную среду, в нашем распоряжении оказывается больше возможных состояний и действий.

In [ ]:
n_states, n_actions = env.observation_space.n, env.action_space.n
print(f"n_states={n_states}, n_actions={n_actions}")

Это определенно много. Слишком много для жесткого программирования, как мы делали с предыдущей задачей. Давайте используем метод перекрестной энтропии для решения этой задачи.

### Create stochastic policy

На этот раз нашей политикой должно быть распределение вероятностей.

`policy[s, a] = P(выполнить действие a | в состоянии s)`

Поскольку мы по-прежнему используем целочисленные представления состояния и действия, вы можете использовать двумерный массив для представления политики.

Пожалуйста, инициализируйте политику __единообразно__, то есть вероятности всех действий должны быть равны.

In [ ]:
import numpy as np

def initialize_policy(n_states, n_actions):
    # Create an array to store action probabilities
    
    # Ваш код здесь
    policy = ...
    
    return policy

In [ ]:
policy = initialize_policy(n_states, n_actions)
policy

### Play the game

Давайте поиграем в игру, как и раньше, однако на этот раз мы также будем записывать состояния, действия и награды, чтобы использовать их в цикле тренировок.

In [ ]:
env.step(1)

In [ ]:
def generate_session(env, policy, time_limit=10**4):
    state, _ = env.reset()
    states, actions = [], []
    total_reward = 0.

    for _ in range(time_limit):
        # Choose action based on policy and take it.
        # Ваш код здесь
        action = ...
        new_state, reward, is_done, truncated, _ = env.step(action)

        # Record information we just got from the environment.
        # Ваш код здесь
        
        state = new_state
        if is_done:
            break

    return states, actions, total_reward

In [ ]:
states, actions, reward = generate_session(env, policy)

Давайте посмотрим на первоначальное распределение вознаграждений за нашу "наивную" политику.

In [ ]:
sample_rewards = [generate_session(env, policy, time_limit=1000)[2] for _ in range(200)]
plt.hist(sample_rewards, bins=20)
plt.vlines([np.percentile(sample_rewards, 50)], [0], [100], label="50'th percentile", color="green")
plt.vlines([np.percentile(sample_rewards, 90)], [0], [100], label="90'th percentile", color="red")
plt.legend();

In [ ]:
np.percentile(sample_rewards, 90)

### Crossentropy method step

In [ ]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile):
    """
    Select states and actions from games that have rewards >= percentile.

    Compute minimum reward for session to be elite and choose elite states
    and actions based on this threshold.

    Note that states_batch and actions_batch are both 2d lists, i.e. lists
    containing lists of states and actions from each session in batch.
    """
    
    elite_states = []
    elite_actions = []
    # Ваш код здесь
    
    return elite_states, elite_actions

In [ ]:
def get_new_policy(elite_states, elite_actions):
    """
    Given a list of elite states/actions from select_elites, return a new
    policy where each action probability is proportional to

        policy[s, a] ~ #[occurrences of s and a in elite states/actions]

    Don't forget to normalize the policy to get valid probabilities.
    For states that you never visited, use a uniform distribution.
    """

    new_policy = np.ones([n_states, n_actions])

    # Set probabilities for actions given elite states & actions.
    # Ваш код здесь
    
    return new_policy

### Training loop

Generate sessions, select N best and fit to those.

In [ ]:
def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    A convenience function that displays training progress. 
    No cool math here, just charts.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])
    
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)

    mean_rewards = [mean_reward for mean_reward, threshold in log]
    reward_thresholds = [threshold for mean_reward, threshold in log]
    plt.plot(mean_rewards, label="Mean rewards")
    plt.plot(reward_thresholds, label="Reward thresholds")
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines(
        [np.percentile(rewards_batch, percentile)],
        ymin=[0],
        ymax=[100],
        label="percentile",
        color="red",
    )
    plt.legend()
    plt.grid()
    clear_output(wait=True)
    print(f"mean reward = {mean_reward:.3f}, threshold={threshold:.3f}")
    plt.show()

In [ ]:
# reset policy
policy = initialize_policy(n_states, n_actions)

In [ ]:
n_sessions = 250     # sample this many sessions
percentile = 70      # take this percent of session with highest rewards
learning_rate = 0.8  # how quickly the policy is updated, on a scale from 0 to 1

log = []

for i in range(40):
    # Generate a list of n_sessions new sessions, select elites and compute
    # new policy based on them. After that update the existing policy wrt
    # learning rate.
    sessions = [generate_session(env, policy)  for _ in range(n_sessions)]
    states_batch = [session_states for session_states, session_actions, session_reward in  sessions]
    actions_batch = [session_actions for session_states, session_actions, session_reward in  sessions]
    rewards_batch = [session_reward for session_states, session_actions, session_reward in  sessions]

    # Ваш код здесь
    
    # display results on chart
    show_progress(rewards_batch, log, percentile)

### Анализ результатов

Возможно, вы заметили, что проблема с такси быстро переходит от очень низких значений к почти оптимальным, а затем возвращается обратно. Это вызвано (по крайней мере, частично) врожденной случайностью окружающей среды. А именно, исходные точки взаимодействия пассажира и водителя меняются от эпизода к эпизоду.

В таком случае, если кроссэнтропийная политика не смогла научиться побеждать с одной четкой отправной точки, она просто откажется от нее, потому что никакие занятия с этой отправной точки не приведут ее в "элиту".

Чтобы смягчить эту проблему, вы можете либо снизить порог для элитных сессий (метод клейкой ленты), либо изменить способ оценки стратегии (теоретически правильный способ). Для каждого начального состояния вы можете выбрать действие случайным образом, а затем оценить это действие, запустив несколько игр, начиная с него, и усреднив общую награду. Выбор элитных сессий с такой выборкой (где вознаграждение за каждую сессию рассчитывается как среднее значение вознаграждений за все сессии с одинаковым начальным состоянием и действием) должен повысить эффективность вашей политики.

## Копаем глубже: приблизительная кроссэнтропия с помощью нейронных сетей

В этом разделе мы расширим вашу реализацию CEM с помощью нейронных сетей! Вы обучите многослойную нейронную сеть решать простые игры с непрерывным пространством состояний.

![img](https://watanimg.elwatannews.com/old_news_images/large/249765_Large_20140709045740_11.jpg)

In [ ]:
# .env is to remove auto-assigned time limit wrapper
env = gym.make("CartPole-v0", render_mode='rgb_array').env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]

print("state vector dim =", state_dim)
print("n_actions =", n_actions)

plt.imshow(env.render())
plt.show()

In [ ]:
env.reset()

Здесь, как и в ```MountainCar-v0```, мы будем управлять тележкой, которую мы можем перемещать вправо или влево. Однако наша цель здесь иная. В этой среде мы хотим как можно дольше удерживать шест, прикрепленный к верхней части нашей тележки, от падения.

### Neural Network Policy

Для этого задания мы будем использовать упрощенную реализацию нейронной сети из [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html). Вот что вам понадобится:
* `agent.partial_fit(states, actions)` - выполняет однократный тренировочный проход по данным, чтобы увеличить вероятность выполнения заданных "actions" в заданных "states".
* `agent.predict_proba(states)` - предсказывает вероятности всех действий, используя матрицу вида `[len(states), n_actions] `

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
[env.reset()] * n_actions

In [ ]:
n_actions

In [ ]:
agent = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation="tanh",
    warm_start=True,
    max_iter=1
)

# initialize agent to the dimension of state space and number of actions
agent.fit([env.reset()[0]] * n_actions, range(n_actions))

In [ ]:
agent.predict_proba([env.reset()[0]]).squeeze()

Несмотря на очевидные различия, вы обнаружите, что процедура обучения для такого агента очень похожа на ту, которую мы использовали в предыдущей части. Нам даже не нужно будет переписывать большинство наших вспомогательных функций! Однако кое-что изменилось - это способ получения вероятностей действий. Итак, давайте адаптируем нашу функцию `generate_session` к этой новой политике, основанной на агентах.

In [ ]:
def generate_session(env, agent, time_limit=10**4):
    state, _ = env.reset()
    states, actions = [], []
    total_reward = 0.
    for _ in range(time_limit):
        # Use agent to predict a vector of action probabilities for current 
        # state and use the probabilities you predicted to pick an action.
        # Sample actions, don't just take the most likely one!

        # Ваш код здесь
        action = ...
        
        states.append(state)
        actions.append(action)
        
        state, reward, is_done, truncated, _ = env.step(action)
        total_reward += reward

        # Record information we just got from the environment.
        if is_done:
            break

    return states, actions, total_reward

In [ ]:
states, actions, reward = generate_session(env, agent, time_limit=100)
print("states:", np.stack(states))
print("actions:", actions)
print("reward:", reward)

### Training loop

In [ ]:
n_sessions = 100
percentile = 70

log = []

for _ in range(100):
    # Generate new sessions, select elites and update our agent.

    sessions = [generate_session(env, agent)  for _ in range(n_sessions)]
    states_batch = [session_states for session_states, session_actions, session_reward in  sessions]
    actions_batch = [session_actions for session_states, session_actions, session_reward in  sessions]
    rewards_batch = [session_reward for session_states, session_actions, session_reward in  sessions]

    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

    # Ваш код здесь
    ...

    show_progress(
        rewards_batch, 
        log, 
        percentile, 
        reward_range=[0, np.max(rewards_batch)]
    )

    if np.mean(rewards_batch) > 190:
        print("You Win! You may stop training now via KeyboardInterrupt.")
        break

### Analysing the results

In [ ]:
import gym.wrappers

with gym.wrappers.RecordVideo(
    gym.make("CartPole-v0", render_mode='rgb_array'),
    video_folder="videos", 
    episode_trigger=lambda x: True
    ) as env_monitor:
    generate_session(env_monitor, agent)

In [ ]:
import sys
from pathlib import Path
from base64 import b64encode
from IPython.display import HTML

video_paths = sorted([file for file in Path("videos").iterdir() if file.suffix == ".mp4"])
video_path = video_paths[-1]  # You can also try other indices

if "google.colab" in sys.modules:
    # https://stackoverflow.com/a/57378660/1214547
    with video_path.open("rb") as fp:
        mp4 = fp.read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
else:
    data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))